In [ ]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
!pip install mysql-connector-python
import mysql.connector
import config
import time
import requests
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS

In [70]:
# create list to hold all data
# all_best_sellers = []

for i in range(18300, 32561, 20):
    params = {'api-key': config.api_key,
                     'offset': i}
    url = 'https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json'
    response = requests.get(url, params = params)
    data = response.json()

    for book in data['results']:
        
        each_book = {}
        
        try:
            each_book['title'] = book['title']
            each_book['author'] = book['author']
            each_book['publisher'] = book['publisher']
            each_book['ISBN'] = book['isbns']
            each_book['publish_date'] = book['ranks_history'][0]['published_date']

            # add the dictionary to master list
            all_best_sellers.append(each_book)

        except IndexError:
            continue

    # print what page we are on for auditing purposes
    print(i)

    # wait so we don't hit the API's per minute call limit
    time.sleep(6)


18300
18320
18340
18360
18380
18400
18420
18440
18460
18480
18500
18520
18540
18560
18580
18600
18620
18640
18660
18680
18700
18720
18740
18760
18780
18800
18820
18840
18860
18880
18900
18920
18940
18960
18980
19000
19020
19040
19060
19080
19100
19120
19140
19160
19180
19200
19220
19240
19260
19280
19300
19320
19340
19360
19380
19400
19420
19440
19460
19480
19500
19520
19540
19560
19580
19600
19620
19640
19660
19680
19700
19720
19740
19760
19780
19800
19820
19840
19860
19880
19900
19920
19940
19960
19980
20000
20020
20040
20060
20080
20100
20120
20140
20160
20180
20200
20220
20240
20260
20280
20300
20320
20340
20360
20380
20400
20420
20440
20460
20480
20500
20520
20540
20560
20580
20600
20620
20640
20660
20680
20700
20720
20740
20760
20780
20800
20820
20840
20860
20880
20900
20920
20940
20960
20980
21000
21020
21040
21060
21080
21100
21120
21140
21160
21180
21200
21220
21240
21260
21280
21300
21320
21340
21360
21380
21400
21420
21440
21460
21480
21500
21520
21540
21560
21580
21600
2162

In [71]:
len(all_best_sellers)

7774

In [ ]:
def get_NYT_ISBNs(books):
    ISBNs = []
    years = [2015, 2016, 2017, 2018, 2019]
    
    for book in books:
        if book['publish_date'] in years:
            ISBN = book['ISBN'][0]['isbn13']
            ISBNs.append(ISBN)
            
    return ISBNs

In [ ]:
get_NYT_ISBNs(all_best_sellers)

# Scrape Goodreads for data on NYT Bestsellers

In [17]:
# set preferences for selenium
driver_options = webdriver.chrome.options.Options()
prefs = {"profile.managed_default_content_settings.images": 2}
driver_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=driver_options)

NameError: name 'webdriver' is not defined

In [ ]:
key: fkrr32n88Wn0Z2oUhDxyA
secret: qHvozGoWQFFa9ePzaUOzsrFHgAspQyqSrHMXjFJRhMw

In [ ]:
# this function takes in a list of ISBNs that were gathered from the NYT API 
# and returns a list of dictionaries, where each dictionary reflects data 
# about the NYT bestseller

list_of_NYT_books = []

def get_NYTbook_info(ISBNs):
    """
    get_NYTbook_info():
    Function returns a list of dictionaries with features describing each NYT bestseller
    Params:
        ISBNs: list of ISBNs for each NYT bestseller
    Returns:
        List of dictionaries describing bestselling books
    """

    for ISBN in ISBNs:
        
        # log into goodreads website 
        driver.get('https://www.goodreads.com/')
        
        # wait for website to load
        time.sleep(2)
        driver.find_element_by_id('userSignInFormEmail').send_keys(config.email_address)
        driver.find_element_by_id('user_password').send_keys(config.pw, Keys.ENTER)
        time.sleep(2)
        
        try:
            # get the webpage for each ISBN 
            driver.find_element_by_xpath('/html/body/div[4]/main/div[1]/section[1]/div/div/footer/div[1]/div/form/input').send_keys(ISBN,Keys.ENTER)
            time.sleep(2) 

            # grab the current url to scrape
            url = driver.current_url
            
        # Beautiful soup to scrape each book page for features
            html_page = requests.get(url)
            soup = bs(html_page.content, 'html.parser')

            book_dict = {}

            book_dict['title'] = good.get_title(soup)
            book_dict['ISBN'] = ISBN
            book_dict['author'] = good.get_author(soup)
            book_dict['series'] = good.get_series(soup)
            book_dict['genre'] = good.get_genre(soup)
            book_dict['rating'] = good.get_rating(soup)
            book_dict['publish_date'] = good.get_publish_date(soup)
            book_dict['publish_company'] = good.get_publishing_company(soup)
            book_dict['number_of_pages'] = good.get_pages(soup)
            book_dict['format'] = good.get_format(soup)

            list_of_NYT_books.append(book_dict)

        except AttributeError:
            continue
        
        time.sleep(2)
        

In [ ]:
def get_NYTbook_data(get_NYT_ISBNs())

In [ ]:
import sqlalchemy
import pyodbc
engine = sqlalchemy.create_engine("mssql+pyodbc://<username>:<password>@<dsnname>")

# write the DataFrame to a table in the sql database
df.to_sql("table_name", engine)

# Insert into MySQL

In [57]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password)
    
cursor = cnx.cursor()

In [58]:
#create database
from mysql.connector import errorcode
db_name = 'NYT'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database NYT does not exist.
Database NYT created successfully.


In [59]:
#specify database for connection
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "NYT")

cursor = cnx.cursor()

#create table
DB_NAME = 'NYT'

create_query = """
      CREATE TABLE bestsellers
    ( title VARCHAR(100),
      author VARCHAR(100),
      publisher VARCHAR(100),
      ISBN VARCHAR(30),
      publish_date DATE
      )
    """


In [ ]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "NYT")

cursor = cnx.cursor()

def insert_data(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO bestsellers (ActivityIdentifier, 
                                                ActivityLatitude, 
                                                ActivityLongitude, 
                                                AnalysisStartDate,
                                                Contaminant,
                                                DetectedLevel,
                                                LevelUnit,
                                                ActivityMediaSubdivisionName
                                                ) 
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [ ]:
insert_data(df)